<a href="https://colab.research.google.com/github/Luz-Aquino/chatbot-supermercado-ia/blob/main/Codigo_Supermercado_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio thefuzz groq

import sqlite3
import pandas as pd
from thefuzz import process
import gradio as gr
from groq import Groq
import re
from datetime import datetime

# --- 🔑 API KEY ---
API_KEY = "PEGÁ_TU_CLAVE_ACÁ"

try:
    client = Groq(api_key=API_KEY)
except:
    pass

# --- 1. BASE DE DATOS ---
conn = sqlite3.connect(':memory:', check_same_thread=False)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS usuarios')
cursor.execute('''CREATE TABLE usuarios (dni TEXT PRIMARY KEY, password TEXT, nombre TEXT, apellido TEXT, mail TEXT, fecha_nac TEXT, direccion TEXT)''')
cursor.execute('DROP TABLE IF EXISTS productos')
cursor.execute('''CREATE TABLE productos (id INTEGER PRIMARY KEY, nombre TEXT, marca TEXT, categoria TEXT, precio REAL, stock INTEGER, unidad_medida TEXT, promocion TEXT)''')

datos_super = [
    (1, 'Leche Entera', 'La Serenísima', 'Lacteos', 1200.0, 50, '1 Litro', 'Llevando 3 pagas 2'),
    (2, 'Arroz Oro', 'Gallo', 'Almacen', 1850.5, 100, '1 kg', None),
    (3, 'Gaseosa Cola Original', 'Coca Cola', 'Bebidas', 2400.0, 40, '2.25 L', '2da al 50%'),
    (4, 'Fernet', 'Branca', 'Bebidas', 9500.0, 5, '750 ml', '10% OFF con App'),
    (5, 'Jabón Liquido', 'Ariel', 'Limpieza', 8200.0, 20, '3 L', None),
    (6, 'Yerba Mate', 'Playadito', 'Almacen', 3800.0, 60, '1 kg', 'Precio Cuidado'),
    (7, 'Fideos Tirabuzón', 'Matarazzo', 'Almacen', 1100.0, 80, '500 g', '3x2'),
    (8, 'Aceite Girasol', 'Natura', 'Almacen', 2100.0, 45, '1.5 L', None),
    (9, 'Azúcar Clásica', 'Ledesma', 'Almacen', 950.0, 90, '1 kg', None),
    (10, 'Harina 0000', 'Pureza', 'Almacen', 850.0, 70, '1 kg', None),
    (11, 'Puré de Tomate', 'Arcor', 'Almacen', 700.0, 100, '520 g', '4x3'),
    (12, 'Galletitas Rellenas', 'Oreo', 'Almacen', 1200.0, 150, '117 g', '2x1'),
    (13, 'Queso Cremoso', 'Cremón', 'Lacteos', 5500.0, 30, 'Por kg', None),
    (14, 'Manteca', 'La Paulina', 'Lacteos', 2200.0, 40, '200 g', None),
    (15, 'Yogur Bebible', 'Ser', 'Lacteos', 1500.0, 35, '1 kg', '20% OFF'),
    (16, 'Dulce de Leche', 'La Colonial', 'Almacen', 2800.0, 50, '400 g', None),
    (17, 'Cerveza Rubia', 'Quilmes', 'Bebidas', 1800.0, 120, '1 L', 'Pack x6 $9000'),
    (18, 'Vino Malbec', 'Rutini', 'Bebidas', 12500.0, 10, '750 ml', '15% OFF Caja'),
    (19, 'Agua Mineral', 'Villavicencio', 'Bebidas', 900.0, 200, '1.5 L', None),
    (20, 'Jugo Naranja', 'Citric', 'Bebidas', 2600.0, 25, '1 L', None),
    (21, 'Lavandina', 'Ayudin', 'Limpieza', 1200.0, 60, '1 L', None),
    (22, 'Detergente', 'Magistral', 'Limpieza', 2300.0, 50, '500 ml', '2x1'),
    (23, 'Papel Higiénico', 'Higienol', 'Limpieza', 3500.0, 40, '4 rollos', None),
    (24, 'Shampoo', 'Pantene', 'Perfumeria', 4200.0, 30, '400 ml', '25% OFF 2da unidad'),
    (25, 'Desodorante', 'Dove', 'Perfumeria', 2800.0, 45, '150 ml', None),
    (26, 'Pan Lactal', 'Artesano', 'Panaderia', 3200.0, 25, '500 g', None),
    (27, 'Huevos Blancos', 'Avicola', 'Frescos', 2800.0, 40, '12 un', 'Precio Amigo'),
    (28, 'Pechuga de Pollo', 'Granja Tres Arroyos', 'Carniceria', 6500.0, 20, 'Por kg', None),
    (29, 'Carne Picada', 'Frigorífico', 'Carniceria', 7200.0, 15, 'Por kg', None),
    (30, 'Manzanas Rojas', 'Moño Azul', 'Verduleria', 1800.0, 30, 'Por kg', 'Oferta de Estación')
]
cursor.executemany('INSERT INTO productos VALUES (?,?,?,?,?,?,?,?)', datos_super)
cursor.execute("INSERT INTO usuarios VALUES ('12345678', 'Luz12345', 'Luz', 'Aquino', 'luz@test.com', '01/01/1990', 'Calle Falsa 123')")
conn.commit()

# --- HELPER FORMATO ARG ---
def formato_ar(valor):
    try:
        if valor == "-": return "-"
        val = float(valor)
        formato_us = f"{val:,.2f}"
        return formato_us.replace(",", "X").replace(".", ",").replace("X", ".")
    except:
        return valor

# --- HELPERS UI (CATEGORIAS) ---
def obtener_categorias():
    df = pd.read_sql_query("SELECT DISTINCT categoria FROM productos", conn)
    return ["🔥 Ofertas del Día", "📂 Ver Todo"] + sorted(df['categoria'].tolist())

def filtrar_por_categoria(categoria_elegida):
    if not categoria_elegida: return gr.update(choices=[])

    if categoria_elegida == "🔥 Ofertas del Día":
        query = "SELECT nombre, marca, precio FROM productos WHERE promocion IS NOT NULL"
    elif categoria_elegida == "📂 Ver Todo":
        query = "SELECT nombre, marca, precio FROM productos"
    else:
        query = f"SELECT nombre, marca, precio FROM productos WHERE categoria = '{categoria_elegida}'"

    df = pd.read_sql_query(query, conn)
    # Ordenar alfabéticamente para que se vea ordenado
    df = df.sort_values(by='nombre')

    lista = []
    for i, row in df.iterrows():
        precio_fmt = formato_ar(row['precio'])
        lista.append(f"{row['nombre']} {row['marca']} (${precio_fmt})")
    return gr.update(choices=lista, value=None)

# --- HELPER TEXTO ---
def formato_num(n):
    return int(n) if n == int(n) else n

def formatear_cantidad_texto(cantidad, unidad_medida):
    n_limpio = formato_num(cantidad)
    if "kg" in unidad_medida.lower():
        return f"{n_limpio} kg"

    palabra = "unidad" if n_limpio == 1 else "unidades"
    return f"{n_limpio} {palabra}"

# --- VISUAL: CARTEL INTELIGENTE ---
def generar_html_aviso(stock_restante, promocion, unidad_medida):
    msgs = []

    # 1. Stock Virtual
    if stock_restante <= 0:
        return f"""
        <div style='background-color: #cce5ff; color: #004085; padding: 10px; border-radius: 5px; border: 1px solid #b8daff; text-align: center; font-size: 14px;'>
        ✅ ¡Tenés todo el stock disponible en tu carrito!
        <br>{f"🔥 {promocion}" if promocion else ""}
        </div>
        """
    elif stock_restante < 10:
        txt_stock = formatear_cantidad_texto(stock_restante, unidad_medida)
        sufijo = "disponible" if stock_restante == 1 else "disponibles"
        msgs.append(f"📦 Quedan {txt_stock} {sufijo}")

    # 2. Oferta
    if promocion:
        msgs.append(f"🔥 ¡OFERTA! {promocion}")

    if msgs:
        texto_html = "<br>".join(msgs)
        return f"""<div style='background-color: #fff3cd; color: #000000 !important; padding: 10px; border-radius: 5px; border: 1px solid #ffeeba; text-align: center; font-size: 14px;'>{texto_html}</div>"""
    return ""

def actualizar_info_producto(producto_str, carrito_actual):
    if not producto_str: return gr.update(label="Cant"), ""

    nombre_limpio = producto_str.split(" ($")[0]
    df = pd.read_sql_query(f"SELECT unidad_medida, promocion, stock FROM productos WHERE nombre || ' ' || marca = '{nombre_limpio}'", conn)

    if not df.empty:
        row = df.iloc[0]
        es_pesable = "Por kg" in row['unidad_medida']
        label_cant = "Peso (Kg)" if es_pesable else "Cantidad"
        step = 0.1 if es_pesable else 1

        cant_en_carrito = 0
        if carrito_actual:
            for item in carrito_actual:
                if item[0] == f"{nombre_limpio}":
                    try:
                        cant_limpia = str(item[1]).replace(" kg", "")
                        cant_en_carrito = float(cant_limpia)
                    except: pass
                    break

        stock_restante = row['stock'] - cant_en_carrito
        html_aviso = generar_html_aviso(stock_restante, row['promocion'], row['unidad_medida'])
        return gr.update(label=label_cant, value=1, step=step), html_aviso
    return gr.update(label="Cant"), ""

LISTA_CATEGORIAS_UI = obtener_categorias()

# --- LÓGICA PRECIOS ---
def calcular_precio_final(precio_unitario, cantidad, texto_promo):
    aviso = ""
    precio_full = precio_unitario * cantidad
    subtotal = precio_full

    if not texto_promo: return subtotal, 0, ""
    texto_promo = texto_promo.lower()

    if "3 pagas 2" in texto_promo or "3x2" in texto_promo:
        grupos = int(cantidad) // 3
        sobra = int(cantidad) % 3
        subtotal = ((grupos * 2) + sobra) * precio_unitario
        if grupos > 0: aviso = "🔥 ¡Promo 3x2 Aplicada!"
    elif "2x1" in texto_promo:
        grupos = int(cantidad) // 2
        sobra = int(cantidad) % 2
        subtotal = (grupos + sobra) * precio_unitario
        if grupos > 0: aviso = "🔥 ¡Promo 2x1 Aplicada!"
    elif "% off" in texto_promo and not "caja" in texto_promo:
        nums = re.findall(r'\d+', texto_promo)
        if nums:
            factor = (100 - int(nums[0])) / 100
            subtotal = precio_full * factor
            aviso = f"🔥 Descuento aplicado"
    elif "caja" in texto_promo or "pack" in texto_promo:
        nums = re.findall(r'\d+', texto_promo)
        if cantidad >= 6:
            if nums:
                factor = (100 - int(nums[0])) / 100
                subtotal = precio_full * factor
                aviso = f"🔥 Descuento por Caja/Pack aplicado."
            elif "pack x6" in texto_promo and "$" in texto_promo:
                precios_fijos = re.findall(r'\d+', texto_promo.replace("$", ""))
                if precios_fijos:
                    precio_pack = float(precios_fijos[-1])
                    packs = int(cantidad) // 6
                    sobra = int(cantidad) % 6
                    subtotal = (packs * precio_pack) + (sobra * precio_unitario)
                    aviso = "🔥 Precio Pack Aplicado."
    elif "2da al 50" in texto_promo:
        pares = int(cantidad) // 2
        sobra = int(cantidad) % 2
        precio_par = precio_unitario * 1.5
        subtotal = (pares * precio_par) + (sobra * precio_unitario)
        if pares > 0: aviso = "🔥 Promo 2da al 50%."

    return subtotal, subtotal - precio_full, aviso

# --- HELPER: GENERAR TABLA VISUAL (FORMATO ARG) ---
def generar_tabla_visual(lista_estado):
    lista_visual = []
    for item in lista_estado:
        # item: [Nombre, CantFmt, PrecioRaw, DescRaw, SubtotalRaw]
        nombre = item[0]
        cant = item[1]

        precio_fmt = f"${formato_ar(item[2])}"

        desc_val = item[3]
        desc_fmt = f"${formato_ar(desc_val)}" if isinstance(desc_val, (int, float)) else "-"

        subtotal_fmt = f"${formato_ar(item[4])}"

        lista_visual.append([nombre, cant, precio_fmt, desc_fmt, subtotal_fmt])
    return lista_visual

# --- CARRITO ---
def agregar_al_carrito(producto_str, cantidad_input, carrito_actual):
    try:
        # 1. RECUPERAR DATOS
        nombre_limpio = producto_str.split(" ($")[0] if producto_str else ""
        if not producto_str:
            return carrito_actual, generar_tabla_visual(carrito_actual), gr.update(), "### 💰 Total a Pagar: $0,00", ""

        cant_real = float(cantidad_input) if cantidad_input else 0
        if cant_real == 0:
            total_actual = sum(i[4] for i in carrito_actual)
            return carrito_actual, generar_tabla_visual(carrito_actual), gr.update(), f"### 💰 Total a Pagar: ${formato_ar(total_actual)}", "⚠️ Cantidad neutra."

        df = pd.read_sql_query(f"SELECT * FROM productos WHERE nombre || ' ' || marca = '{nombre_limpio}'", conn)
        row = df.iloc[0]
        stock_real = row['stock']
        es_pesable = "Por kg" in row['unidad_medida']

        # 2. BUSCAR EN CARRITO (State)
        indice = -1
        cant_ant = 0
        nuevo_carrito = list(carrito_actual)

        if carrito_actual:
            for i, item in enumerate(carrito_actual):
                if item[0] == f"{row['nombre']} {row['marca']}":
                    indice = i
                    cant_limpia = str(item[1]).replace(" kg", "")
                    cant_ant = float(cant_limpia)
                    break

        # 3. VALIDAR STOCK
        nueva_cant = cant_ant + cant_real
        total_actual = sum(i[4] for i in carrito_actual)

        if cant_real > 0 and nueva_cant > stock_real:
             stock_restante_real = stock_real - cant_ant
             txt_stock_real = int(stock_restante_real) if stock_restante_real == int(stock_restante_real) else stock_restante_real

             html_error = f"""<div style='background-color: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px; text-align: center; font-weight: bold;'>⛔ Solo quedan {txt_stock_real} disponibles.</div>"""

             html_amarillo = generar_html_aviso(stock_restante_real, row['promocion'], row['unidad_medida'])
             return carrito_actual, generar_tabla_visual(carrito_actual), html_amarillo, f"### 💰 Total a Pagar: ${formato_ar(total_actual)}", html_error

        # 4. BORRAR
        if nueva_cant <= 0.001:
            if indice != -1:
                nuevo_carrito.pop(indice)
                total = sum(i[4] for i in nuevo_carrito)
                html_amarillo = generar_html_aviso(stock_real, row['promocion'], row['unidad_medida'])
                html_feedback = f"""<div style='background-color: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px; text-align: center;'>🗑️ {row['nombre']} eliminado.</div>"""
                return nuevo_carrito, generar_tabla_visual(nuevo_carrito), html_amarillo, f"### 💰 Total a Pagar: ${formato_ar(total)}", html_feedback

        # 5. ACTUALIZAR
        subtotal, descuento_nuevo, aviso_oferta = calcular_precio_final(row['precio'], nueva_cant, row['promocion'])

        # Guardamos en STATE los valores crudos
        cant_fmt = f"{nueva_cant:.3f} kg" if es_pesable else f"{int(nueva_cant)}"

        item = [f"{row['nombre']} {row['marca']}", cant_fmt, row['precio'], descuento_nuevo if descuento_nuevo < 0 else "-", subtotal]

        if indice != -1: nuevo_carrito[indice] = item
        else: nuevo_carrito.append(item)

        total = sum(i[4] for i in nuevo_carrito)

        # 6. MENSAJES Y SALIDA
        stock_restante_nuevo = stock_real - nueva_cant
        html_amarillo = generar_html_aviso(stock_restante_nuevo, row['promocion'], row['unidad_medida'])

        descuento_anterior = 0
        if indice != -1 and carrito_actual:
             try:
                 val_ant = carrito_actual[indice][3]
                 if isinstance(val_ant, (int, float)): descuento_anterior = val_ant
             except: pass

        if cant_real < 0:
            cant_abs = abs(cant_real)
            txt_resta = formatear_cantidad_texto(cant_abs, row['unidad_medida'])
            html_feedback = f"""<div style='background-color: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px; text-align: center;'>📉 Restaste {txt_resta}</div>"""
        elif descuento_nuevo < descuento_anterior:
            html_feedback = f"""<div style='background-color: #d4edda; color: #155724; padding: 10px; border-radius: 5px; font-weight: bold; text-align: center;'>✅ {aviso_oferta}</div>"""
        else:
            html_feedback = f"""<div style='background-color: #cce5ff; color: #004085; padding: 10px; border-radius: 5px; text-align: center;'>✔️ Agregado.</div>"""

        return nuevo_carrito, generar_tabla_visual(nuevo_carrito), html_amarillo, f"### 💰 Total a Pagar: ${formato_ar(total)}", html_feedback

    except Exception as e:
        print(e)
        return carrito_actual, generar_tabla_visual(carrito_actual), gr.update(), "Error", f"❌ Error: {str(e)}"

def limpiar_carrito(): return [], [], "", "### 💰 Total a Pagar: $0,00", ""

# --- VALIDACIONES Y AUTH (Igual) ---
def convertir_fecha(dia, mes, anio):
    try:
        meses = {"Enero":1, "Febrero":2, "Marzo":3, "Abril":4, "Mayo":5, "Junio":6, "Julio":7, "Agosto":8, "Septiembre":9, "Octubre":10, "Noviembre":11, "Diciembre":12}
        return True, datetime(int(anio), meses[mes], int(dia))
    except: return False, None
def validar_formato_dni(dni): return bool(re.match(r'^\d{7,8}$', dni))
def validar_seguridad_password(password): return len(password) >= 8 and re.search(r"[A-Z]", password) and re.search(r"[0-9]", password)
def registrar_usuario(nombre, apellido, dni, mail, dia, mes, anio, direccion, password):
    if not all([nombre, apellido, dni, mail, dia, mes, anio, password]): return "⚠️ Faltan campos (*)"
    if not validar_formato_dni(dni): return "⚠️ DNI inválido."
    ok, f_obj = convertir_fecha(dia, mes, anio)
    if not ok: return "⚠️ Fecha inválida."
    if (datetime.now().year - int(anio)) < 18: return "⛔ Debés ser mayor de 18."
    if not validar_seguridad_password(password): return "⚠️ Contraseña insegura (Min 8, 1 Mayus, 1 Num)."
    try:
        cursor.execute("INSERT INTO usuarios VALUES (?,?,?,?,?,?,?)", (dni, password, nombre, apellido, mail, f_obj.strftime('%d/%m/%Y'), direccion))
        conn.commit()
        return f"✅ ¡Usuario {nombre} registrado!"
    except: return "❌ DNI ya registrado."
def validar_login(dni, password):
    user = cursor.execute("SELECT * FROM usuarios WHERE dni=? AND password=?", (dni, password)).fetchone()
    if user: return f"👋 Hola {user[2]}", gr.update(visible=True), gr.update(visible=False)
    else: return "❌ Datos incorrectos", gr.update(visible=False), gr.update(visible=True)
def restaurar_pass(dni, mail, nueva):
    if not cursor.execute("SELECT * FROM usuarios WHERE dni=? AND mail=?", (dni, mail)).fetchone(): return "❌ No encontrado."
    cursor.execute("UPDATE usuarios SET password=? WHERE dni=?", (nueva, dni))
    conn.commit()
    return "✅ Clave cambiada."
def cerrar(): return [gr.update(visible=False), gr.update(visible=True), "", "", "Cerrado.", [], [], "", "### 💰 Total a Pagar: $0,00", ""]

# --- IA ---
def responder_chat(mensaje):
    try:
        df = pd.read_sql_query("SELECT * FROM productos", conn)
        mensaje = mensaje.lower()
        if "bebida" in mensaje: txt = df[df['categoria']=='Bebidas'].to_string()
        elif "limpieza" in mensaje: txt = df[df['categoria']=='Limpieza'].to_string()
        elif "lacteo" in mensaje or "leche" in mensaje: txt = df[df['categoria']=='Lacteos'][['nombre', 'precio', 'promocion']].to_string()
        elif "almacen" in mensaje: txt = df[df['categoria']=='Almacen'].to_string()
        else:
            opciones = (df['marca'] + ' ' + df['nombre']).tolist()
            mejor, score = process.extractOne(mensaje, opciones)
            txt = df.iloc[opciones.index(mejor)].to_string() if score > 55 else "Nada encontrado."
        client = Groq(api_key=API_KEY)
        chat = client.chat.completions.create(messages=[{"role": "user", "content": f"Usuario: {mensaje}. Data: {txt}. Si hay promo avisale."}], model="llama-3.3-70b-versatile")
        return chat.choices[0].message.content
    except Exception as e: return f"⚠️ Error IA: {str(e)}"

# --- INTERFAZ ---
with gr.Blocks(theme=gr.themes.Soft()) as app:
    carrito_state = gr.State([])

    with gr.Group() as bloque_acceso:
        with gr.Tab("🔐 Login"):
            dni_input = gr.Textbox(label="DNI *")
            pass_input = gr.Textbox(label="Contraseña *", type="password")
            login_btn = gr.Button("Ingresar", variant="primary")
            login_output = gr.Textbox(label="Estado", interactive=False)
        with gr.Tab("📝 Registro"):
             with gr.Row():
                reg_nombre = gr.Textbox(label="Nombre *")
                reg_apellido = gr.Textbox(label="Apellido *")
             with gr.Row():
                reg_dni = gr.Textbox(label="DNI *")
                reg_mail = gr.Textbox(label="Email *")
             with gr.Row():
                dd_dia = gr.Dropdown(choices=[str(i) for i in range(1, 32)], label="Fecha Nacimiento: Día *")
                dd_mes = gr.Dropdown(choices=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"], label="Mes *")
                dd_anio = gr.Dropdown(choices=[str(i) for i in range(1940, 2026)], label="Año *")
             reg_dir = gr.Textbox(label="Dirección")
             reg_pass = gr.Textbox(label="Contraseña *", type="password")
             reg_btn = gr.Button("Crear Cuenta")
             reg_output = gr.Textbox(label="Resultado")
        with gr.Tab("🔄 Restaurar"):
            res_dni = gr.Textbox(label="DNI")
            res_mail = gr.Textbox(label="Email")
            res_new_pass = gr.Textbox(label="Nueva Contraseña", type="password")
            res_btn = gr.Button("Cambiar")
            res_output = gr.Textbox(label="Estado")

    with gr.Group(visible=False) as bloque_principal:
        with gr.Row():
            gr.Markdown("## 👋 Bienvenido a tu compra")
            logout_btn = gr.Button("🚪 Salir", size="sm", variant="stop")

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 🤖 Asistente IA")
                chatbot_input = gr.Textbox(label="Consultar")
                chatbot_btn = gr.Button("Enviar")
                chatbot_output = gr.Markdown()

            with gr.Column(scale=2):
                gr.Markdown("### 🛒 Agregar al Carrito")

                with gr.Row():
                    cat_dropdown = gr.Dropdown(choices=LISTA_CATEGORIAS_UI, label="Categoría", scale=2)
                    prod_dropdown = gr.Dropdown(choices=[], label="Producto", scale=3)
                    cant_num = gr.Number(value=1, label="Cant/Peso (+Agregar / -Sacar)", precision=1, scale=1)

                add_btn = gr.Button("🔄 Actualizar Carrito (Sumar/Restar)", variant="primary", size="lg")

                info_promo_ui = gr.HTML("")

                carrito_ui = gr.Dataframe(
                    headers=["Producto", "Cant/Peso", "Precio Unit.", "Descuento", "Subtotal"],
                    datatype=["str", "str", "str", "str", "str"],
                    label="Detalle de Compra",
                    interactive=False
                )

                aviso_lbl = gr.HTML("")

                total_txt = gr.Markdown("### 💰 Total a Pagar: $0,00")
                clean_btn = gr.Button("🗑️ Vaciar Todo", variant="secondary")

    # Eventos
    login_btn.click(fn=validar_login, inputs=[dni_input, pass_input], outputs=[login_output, bloque_principal, bloque_acceso])
    pass_input.submit(fn=validar_login, inputs=[dni_input, pass_input], outputs=[login_output, bloque_principal, bloque_acceso])
    reg_btn.click(fn=registrar_usuario, inputs=[reg_nombre, reg_apellido, reg_dni, reg_mail, dd_dia, dd_mes, dd_anio, reg_dir, reg_pass], outputs=reg_output)
    res_btn.click(fn=restaurar_pass, inputs=[res_dni, res_mail, res_new_pass], outputs=res_output)
    logout_btn.click(fn=cerrar, inputs=None, outputs=[bloque_principal, bloque_acceso, login_output, pass_input, login_output, carrito_state, carrito_ui, info_promo_ui, total_txt, aviso_lbl])

    chatbot_btn.click(fn=responder_chat, inputs=chatbot_input, outputs=chatbot_output)
    chatbot_input.submit(fn=responder_chat, inputs=chatbot_input, outputs=chatbot_output)

    cat_dropdown.change(fn=filtrar_por_categoria, inputs=cat_dropdown, outputs=prod_dropdown)

    prod_dropdown.change(
        fn=actualizar_info_producto,
        inputs=[prod_dropdown, carrito_state],
        outputs=[cant_num, info_promo_ui]
    )

    add_btn.click(fn=agregar_al_carrito, inputs=[prod_dropdown, cant_num, carrito_state], outputs=[carrito_state, carrito_ui, info_promo_ui, total_txt, aviso_lbl])
    clean_btn.click(fn=limpiar_carrito, inputs=None, outputs=[carrito_state, carrito_ui, info_promo_ui, total_txt, aviso_lbl])

app.launch(share=True)